In [1]:
import pandas as pd
import numpy as np

In [2]:
# import transcript data
transcript_df = pd.read_parquet("~/grimoire/this-viz-will-kill-you/data/raw/transcript_df.parquet")

# validate output
print(transcript_df.shape)
display(transcript_df["episode_id"].unique())
display(transcript_df.head())

(63307, 4)


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165])

,episode_id,episode_name,speaker,text
0,1,TPWKY-Episode-95-Tetanus,Erin Welsh,Hey everyone. We're including a content warnin...
1,1,TPWKY-Episode-95-Tetanus,Erin Allmann Updyke,"""Mrs. Cyril, Orchard Street, age 24 was delive..."
2,1,TPWKY-Episode-95-Tetanus,,"The wrinkled forehead, the elevated brow, the ..."
3,1,TPWKY-Episode-95-Tetanus,,"The paroxysm was renewed by a slight noise, th..."
4,1,TPWKY-Episode-95-Tetanus,TPWKY,(This Podcast Will Kill You intro theme)


In [3]:
# remove transitions and into theme
transcript_df = transcript_df.loc[transcript_df["speaker"] != "TPWKY"]

# replace empty strings with nan
transcript_df["speaker"] = transcript_df["speaker"].replace('', np.nan, regex=True)

# fill down speaker names
transcript_df["speaker"].ffill(inplace=True)

# check output
display(transcript_df.head())

,episode_id,episode_name,speaker,text
0,1,TPWKY-Episode-95-Tetanus,Erin Welsh,Hey everyone. We're including a content warnin...
1,1,TPWKY-Episode-95-Tetanus,Erin Allmann Updyke,"""Mrs. Cyril, Orchard Street, age 24 was delive..."
2,1,TPWKY-Episode-95-Tetanus,Erin Allmann Updyke,"The wrinkled forehead, the elevated brow, the ..."
3,1,TPWKY-Episode-95-Tetanus,Erin Allmann Updyke,"The paroxysm was renewed by a slight noise, th..."
5,1,TPWKY-Episode-95-Tetanus,Erin Allmann Updyke,That was really difficult to read.


In [4]:
# copy the dataframe
concat_df = transcript_df.copy()

# group by and concatenate text
concat_df = concat_df.groupby(["episode_id", "episode_name", "speaker"])["text"].agg(lambda x: ' '.join(x)).reset_index()

# check output
display(concat_df.head())

,episode_id,episode_name,speaker,text
0,1,TPWKY-Episode-95-Tetanus,Erin Allmann Updyke,"""Mrs. Cyril, Orchard Street, age 24 was delive..."
1,1,TPWKY-Episode-95-Tetanus,Erin Welsh,Hey everyone. We're including a content warnin...
2,2,TPWKY-Bonus-Episode-Ed-Yong,Ed Yong,I love this for them. The short answer is from...
3,2,TPWKY-Bonus-Episode-Ed-Yong,Erin Welsh,"Hi, I'm Erin Welsh and this is This Podcast Wi..."
4,3,TPWKY-Episode-88-Endometriosis,Erin Allmann Updyke,"Yeah, I can't. Wow. Thank you. And I'm Erin Al..."


In [5]:
# count the number of words per speaker
concat_df["num_words"] = concat_df["text"].str.split().str.len()

# count the number of unique words spoken by each speaker
concat_df["num_unique_words"] = concat_df["text"].str.split().apply(set).str.len()

# count the number of questions each speaker asks
concat_df["num_questions"] = concat_df["text"].str.count("\?")

# check output
display(concat_df.head())

,episode_id,episode_name,speaker,text,num_words,num_unique_words,num_questions
0,1,TPWKY-Episode-95-Tetanus,Erin Allmann Updyke,"""Mrs. Cyril, Orchard Street, age 24 was delive...",4713,1449,14
1,1,TPWKY-Episode-95-Tetanus,Erin Welsh,Hey everyone. We're including a content warnin...,5008,1675,26
2,2,TPWKY-Bonus-Episode-Ed-Yong,Ed Yong,I love this for them. The short answer is from...,6196,1557,24
3,2,TPWKY-Bonus-Episode-Ed-Yong,Erin Welsh,"Hi, I'm Erin Welsh and this is This Podcast Wi...",2041,809,26
4,3,TPWKY-Episode-88-Endometriosis,Erin Allmann Updyke,"Yeah, I can't. Wow. Thank you. And I'm Erin Al...",7510,1825,59


In [6]:
# copy concat_df again
totals_df = concat_df.copy()

# calculate the average word length for each speaker
totals_df['avg_word_length'] = totals_df['text'].apply(lambda x: sum(len(word) for word in x.split()) / len(x.split()) if len(x.split()) > 0 else 0)

# calculate total characters and total words for each episode
totals_df = totals_df.groupby(['episode_id', 'speaker']).agg({
    'text': lambda x: ' '.join(x).replace(' ', ''),  # remove spaces to count characters
}).reset_index()

totals_df['total_characters'] = totals_df['text'].str.len()
totals_df['total_words'] = concat_df.groupby(['episode_id', 'speaker'])['text'].apply(lambda x: sum(len(word.split()) for word in x)).reset_index()['text']

# calculate true average word length for each episode
totals_df['true_avg_word_length'] = np.round(totals_df['total_characters'] / totals_df['total_words'], 2)

# check output
display(totals_df.head())

,episode_id,speaker,text,total_characters,total_words,true_avg_word_length
0,1,Erin Allmann Updyke,"""Mrs.Cyril,OrchardStreet,age24wasdeliveredofhe...",22231,4713,4.72
1,1,Erin Welsh,Heyeveryone.We'reincludingacontentwarningforth...,23727,5008,4.74
2,2,Ed Yong,Ilovethisforthem.Theshortanswerisfrommywife.Th...,27570,6196,4.45
3,2,Erin Welsh,"Hi,I'mErinWelshandthisisThisPodcastWillKillYou...",9257,2041,4.54
4,3,Erin Allmann Updyke,"Yeah,Ican't.Wow.Thankyou.AndI'mErinAllmannUpdy...",35860,7510,4.77


In [7]:
# drop excess columns from totals_df
totals_df.drop(["text", "total_characters", "total_words"], axis=1, inplace=True)

# merge concat_df and totals_df
speaker_df = concat_df.merge(totals_df, on=['episode_id', 'speaker'], how='left')

# check output
display(speaker_df.head())

,episode_id,episode_name,speaker,text,num_words,num_unique_words,num_questions,true_avg_word_length
0,1,TPWKY-Episode-95-Tetanus,Erin Allmann Updyke,"""Mrs. Cyril, Orchard Street, age 24 was delive...",4713,1449,14,4.72
1,1,TPWKY-Episode-95-Tetanus,Erin Welsh,Hey everyone. We're including a content warnin...,5008,1675,26,4.74
2,2,TPWKY-Bonus-Episode-Ed-Yong,Ed Yong,I love this for them. The short answer is from...,6196,1557,24,4.45
3,2,TPWKY-Bonus-Episode-Ed-Yong,Erin Welsh,"Hi, I'm Erin Welsh and this is This Podcast Wi...",2041,809,26,4.54
4,3,TPWKY-Episode-88-Endometriosis,Erin Allmann Updyke,"Yeah, I can't. Wow. Thank you. And I'm Erin Al...",7510,1825,59,4.77


In [8]:
# count up the number of "cool"s vs. "interesting"s per speaker
speaker_df = speaker_df.assign(interesting_count=lambda x: x["text"].str.count("interesting"), cool_count=lambda x: x["text"].str.count("cool"))

# calculate the cool:interesting ratio
speaker_df['ratio'] = np.where(
    speaker_df['cool_count'] != 0,
    speaker_df['interesting_count'] / speaker_df['cool_count'],
    np.nan
)

# check output
display(speaker_df.head())

,episode_id,episode_name,speaker,text,num_words,num_unique_words,num_questions,true_avg_word_length,interesting_count,cool_count,ratio
0,1,TPWKY-Episode-95-Tetanus,Erin Allmann Updyke,"""Mrs. Cyril, Orchard Street, age 24 was delive...",4713,1449,14,4.72,7,0,NaN
1,1,TPWKY-Episode-95-Tetanus,Erin Welsh,Hey everyone. We're including a content warnin...,5008,1675,26,4.74,4,6,0.666667
2,2,TPWKY-Bonus-Episode-Ed-Yong,Ed Yong,I love this for them. The short answer is from...,6196,1557,24,4.45,5,0,NaN
3,2,TPWKY-Bonus-Episode-Ed-Yong,Erin Welsh,"Hi, I'm Erin Welsh and this is This Podcast Wi...",2041,809,26,4.54,3,1,3.000000
4,3,TPWKY-Episode-88-Endometriosis,Erin Allmann Updyke,"Yeah, I can't. Wow. Thank you. And I'm Erin Al...",7510,1825,59,4.77,5,0,NaN


In [9]:
# get simple metrics for validation
print(speaker_df.shape)
display(speaker_df["episode_id"].unique())
display(speaker_df.head())

# save to drive
speaker_df.to_parquet("~/grimoire/this-viz-will-kill-you/data/tidy/speaker_df.parquet")

(493, 11)


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165])

,episode_id,episode_name,speaker,text,num_words,num_unique_words,num_questions,true_avg_word_length,interesting_count,cool_count,ratio
0,1,TPWKY-Episode-95-Tetanus,Erin Allmann Updyke,"""Mrs. Cyril, Orchard Street, age 24 was delive...",4713,1449,14,4.72,7,0,NaN
1,1,TPWKY-Episode-95-Tetanus,Erin Welsh,Hey everyone. We're including a content warnin...,5008,1675,26,4.74,4,6,0.666667
2,2,TPWKY-Bonus-Episode-Ed-Yong,Ed Yong,I love this for them. The short answer is from...,6196,1557,24,4.45,5,0,NaN
3,2,TPWKY-Bonus-Episode-Ed-Yong,Erin Welsh,"Hi, I'm Erin Welsh and this is This Podcast Wi...",2041,809,26,4.54,3,1,3.000000
4,3,TPWKY-Episode-88-Endometriosis,Erin Allmann Updyke,"Yeah, I can't. Wow. Thank you. And I'm Erin Al...",7510,1825,59,4.77,5,0,NaN


In [10]:
# send to csv for tableau
speaker_df.to_csv("~/grimoire/this-viz-will-kill-you/data/publish/speaker_df.csv")